# 파이썬 딥러닝 파이토치
## 예제 4-2 : CIFAR-10 데이터를 이용해 Concolutional Neural Network(CNN) 설계하기

In [ ]:
import torch
import torch.nn as nn

In [7]:
class CNN(nn.Module) :
    def __init__(self) :
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels =3,
            out_channels = 8,
            kernel_size =3,
            padding = 1)
        self.conv2 = nn.Conv2d(
            in_channels =8,
            out_channels = 16,
            kernel_size =3,
            padding = 1)
        self.pool = nn.MaxPool2d(
            kernel_size = 2,
            stride =2)
        self.fc1 = nn.Linear(8 * 8 * 16, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)

In [8]:
def forward(self, x) :
    x = self.conv1(x)
    x = F.relu(x)
    x = self.pool(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = self.pool(x)

    x = x.view(-1, 8 * 8 * 16)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.log_softmax(x)
    return x